In [19]:
import os
import random

from PIL import Image
import imagehash

from tqdm.notebook import tqdm

In [31]:
base = "Train/Movie"
classes = ["Full Body Sitting", "Full Body Standing", "Half Body", "Head Only"]

input_directories = [f"../temporal_data_extraction/human_poses_classified/{base}/{class_name}" for class_name in classes]

In [32]:
cutoff = 8
window = 5

In [33]:
def group_similar_images(ordered_dir, cutoff, window):
    image_locs = [f"{ordered_dir}/{file_name}" for file_name in os.listdir(ordered_dir) if file_name.endswith(".jpg")]
    groups = [[image_locs[0]]]
    
    for image_loc in tqdm(image_locs):
        img_hash = imagehash.average_hash(Image.open(image_loc))
        
        closest_group_idx = -1
        closest_group_diff = 65
        
        for offset, group in enumerate(groups[-window:][::-1]):
            group_idx = len(groups) - offset - 1
            last_hash = imagehash.average_hash(Image.open(group[-1]))
            diff = img_hash - last_hash
            
            if diff < closest_group_diff:
                closest_group_idx = group_idx
                closest_group_diff = diff
        
        if closest_group_diff <= cutoff:
            groups[closest_group_idx].append(image_loc)
        else:
            groups.append([image_loc])
    
    return groups

In [34]:
groups = []

for input_directory in input_directories:
    dir_groups = group_similar_images(input_directory, cutoff, window)
    print(len(dir_groups))
    groups += dir_groups

  0%|          | 0/55 [00:00<?, ?it/s]

45


  0%|          | 0/1157 [00:00<?, ?it/s]

251


  0%|          | 0/5331 [00:00<?, ?it/s]

425


  0%|          | 0/3661 [00:00<?, ?it/s]

607


In [35]:
target_test_count = 250
target_train_count = 1200
output_size = 128
min_size = 64
sequence_proportion = 0.05

In [36]:
train_output_directory = f"../temporal_data_extraction/cyclegan/{base}/Train"
test_output_directory = f"../temporal_data_extraction/cyclegan/{base}/Test"

os.makedirs(train_output_directory, exist_ok=False)
os.makedirs(test_output_directory, exist_ok=False)

In [37]:
for group_no, group in enumerate(tqdm(groups)):
    selected_patches = random.sample(range(len(group)), k=max(1, int(sequence_proportion * len(group))))
    
    for i in selected_patches:
        x = Image.open(group[i])
        
        if x.width < min_size or x.height < min_size:
            continue
        
        x = x.resize((output_size, output_size))
        x.save(f"{train_output_directory}/{group_no:05d}_{i:05d}.jpg")

  0%|          | 0/1328 [00:00<?, ?it/s]

In [38]:
valid_train_files = [file_name for file_name in os.listdir(train_output_directory) if file_name.endswith(".jpg")]

In [39]:
len(valid_train_files)

1306

In [40]:
if len(valid_train_files) > target_train_count:
    available_for_test = min(len(valid_train_files) - target_train_count, target_test_count)
    selected_for_test = random.sample(valid_train_files, k=available_for_test)
    
    for file_name in selected_for_test:
        os.rename(f"{train_output_directory}/{file_name}", f"{test_output_directory}/{file_name}")
    
    print("Moved", len(selected_for_test), "files to test")

Moved 106 files to test


In [41]:
valid_train_files = [file_name for file_name in os.listdir(train_output_directory) if file_name.endswith(".jpg")]

In [42]:
len(valid_train_files)

1200

In [43]:
if len(valid_train_files) > target_train_count:
    selected_for_delete = random.sample(valid_train_files, k=len(valid_train_files) - target_train_count)
    
    for file_name in selected_for_delete:
        os.remove(f"{train_output_directory}/{file_name}")
    
    print("Deleted", len(selected_for_delete), "files")